**COMPRESS TIME SERIES**

In [ ]:
import sys, os
from sys import argv
from csv import reader

import keras
import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import models, layers,optimizers, losses, metrics
from keras.models import Model,load_model
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization, Conv1D, MaxPooling1D, UpSampling1D
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

Utility functions to read and preproccess data

In [ ]:
def read_data(dataset_path):
    dataset=open(dataset_path,"r")
    train_data = []
    names = []
    with open(dataset_path, 'r') as read_obj:
        for row in dataset:
            serie=row.split()
            names.append(serie.pop(0)) #save the names of the series 
            serie=[float(x) for x in serie]
            train_data.append(np.array(serie))
    return train_data, names

def scale_data(X):
    X=np.vstack(X)
    scaler = MinMaxScaler()
    X=scaler.fit_transform(X)
    return X

def preprocces_data(data):
    x=data.shape[0]
    y=data.shape[1]
    X = data.reshape((x,y, 1))
    return X,y

Define file paths (given as arguments)
*change the file names to the ones you want to run. 

In [ ]:
# #parse comand line arguments
# dataset_path= ''
# for index, argument in enumerate(argv):
#     if argument == '-d': dataset_path = argv[index+1]
#     if argument == '-od': input_out_path = argv[index+1]
#     if argument == '-oq': query_out_path = argv[index+1]

# if not dataset_path: dataset_path =input('Please provide the file path of the input/query data: ')
# if not input_out_path: input_out_path = input('Please provide the path to store the encoded input data: ')
# if not query_out_path: query_out_path = input('Please provide the path to store the encoded query data: ')
# model_path = input('Please provide the path of the training model: ')

dataset_path = 'nasdaq2007_17.csv'
input_out_path = 'input_encoded.csv'
query_out_path = 'query_encoded.csv'
model_path ='second_best_model.h5'

Proccess data, load model and compress

In [ ]:
#read and prepare data
data,names = read_data(dataset_path) #read file
data = scale_data(data) #scale file data
X,y = preprocces_data(data)#reshape data

#load encoder
AE = load_model(model_path)
encoder_output = AE.get_layer('bottleneck_layer').output

#build a model
encoder_model = keras.Model(AE.input,encoder_output)

encoded_data = encoder_model.predict(X) #encode all data

Save to corresponding files

In [ ]:
final_data = np.column_stack((names, encoded_data))#merge encoded timeseries with their names
input,query = np.split(final_data,[349]) #split the data in input and qywry timeseries to be used in the project2 (349/10)
print("input: ",input.shape)
print("query: ",query.shape)
#save on csv files
np.savetxt("input.csv", input ,fmt='%s', delimiter="\t")
np.savetxt("query.csv", query ,fmt='%s', delimiter="\t")

encoder_model.summary()#print model summary